### ENV Setup

In [1]:
!pip install transformers


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install optuna


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install 'accelerate>=0.26.0'


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install --upgrade pyarrow


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install datasets


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install torch


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


### Implementation

In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from torch.nn.functional import softmax
import numpy as np
from tqdm import tqdm

/Users/vittoriadiachenko/PycharmProjects/knowledge_manipulation/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fork_df = pd.read_csv("/Users/vittoriadiachenko/PycharmProjects/knowledge_manipulation/datasets/extended_fork.csv")

In [3]:
import ast

def is_empty_array(x):
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            return isinstance(parsed, list) and len(parsed) == 0
        except:
            return False
    return False


empty_mask = fork_df["text"].apply(is_empty_array)
print(f"Empty array rows: {empty_mask.sum()} / {len(fork_df)}")

def dedup_text_list(x):
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return str(list(dict.fromkeys(parsed)))
        except:
            return x
    return x

fork_df["text"] = fork_df["text"].apply(dedup_text_list)



def filter_short_texts(x, min_words=5):
    if isinstance(x, str):
        try:
            x = ast.literal_eval(x)
        except:
            return x 
    if not isinstance(x, list):
        return x

    filtered = [s for s in x if isinstance(s, str) and len(s.split()) >= min_words]
    return filtered

fork_df["text"] = fork_df["text"].apply(lambda x: filter_short_texts(x, min_words=3))

Empty array rows: 6516 / 37040


In [4]:
fork_df = fork_df[~empty_mask].reset_index(drop=True)
fork_df = fork_df.drop_duplicates(subset=["text"]).reset_index(drop=True)

In [5]:
print(fork_df["label"].value_counts())
print(fork_df["label"].value_counts(normalize=True) * 100)

label
1    14487
0    14384
Name: count, dtype: int64
label
1    50.17838
0    49.82162
Name: proportion, dtype: float64


In [6]:
import ast

def join_texts(x):
    if isinstance(x, str):
        try:
            x_list = ast.literal_eval(x)
            if isinstance(x_list, list):
                return "\n".join(x_list)
            else:
                return str(x_list)
        except Exception:
            return x
    elif isinstance(x, list):
        return "\n".join(x)
    else:
        return str(x)

fork_df["joined_texts"] = fork_df["text"].apply(join_texts)
fork_df.head()

,Unnamed: 0,page_title,text,label,joined_texts
0,0,Железнодорожный (Нижегородская область),"[В посёлке имеются детский сад, школа, несколь...",1,"В посёлке имеются детский сад, школа, нескольк..."
1,1,Улица Чкалова (Мелитополь),"[Улица Чкалова — улица на севере Мелитополя, и...",1,"Улица Чкалова — улица на севере Мелитополя, ид..."
2,2,Моленар,"[Моленар, Брам (1961—2023) — программист, акти...",1,"Моленар, Брам (1961—2023) — программист, актив..."
3,3,"Чтак, Валерий Сергеевич","[2015 — «Доброе утро, иностранцы», Triangle Ga...",1,"2015 — «Доброе утро, иностранцы», Triangle Gal..."
4,4,GSMA,[Megafon PSSC (Мегафон) (Компания была исключе...,1,Megafon PSSC (Мегафон) (Компания была исключен...


In [7]:
print(len(fork_df))

28871


Fine tuning step

In [8]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from helpers.bert_trainer_helper import BertTrainerHelper

# Split and convert
fork_df_train, fork_df_eval = train_test_split(fork_df, test_size=0.2, stratify=fork_df["label"], random_state=42)
train_ds = Dataset.from_pandas(fork_df_train[["joined_texts", "label"]])
eval_ds = Dataset.from_pandas(fork_df_eval[["joined_texts", "label"]])

# Initialize helper
helper = BertTrainerHelper()

# Tokenize datasets
train_ds = helper.tokenize_dataset(train_ds)
eval_ds = helper.tokenize_dataset(eval_ds)

# Base training args
args = TrainingArguments(
    output_dir="./manipulation_classifier",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_dir="./logs",
    save_total_limit=2,
    bf16=True
)

trainer = Trainer(
    model_init=helper.model_init,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=helper.tokenizer,
    compute_metrics=helper.compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

# Run hyperparameter search
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    n_trials=20,
    hp_space=helper.hp_space_optuna,
    compute_objective=lambda metrics: metrics["eval_f1"]
)

# Retrain with best hyperparameters
best_args = TrainingArguments(
    output_dir="./manipulation_classifier_best",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=best_run.hyperparameters["num_train_epochs"],
    per_device_train_batch_size=best_run.hyperparameters["per_device_train_batch_size"],
    per_device_eval_batch_size=best_run.hyperparameters["per_device_train_batch_size"],
    learning_rate=best_run.hyperparameters["learning_rate"],
    weight_decay=best_run.hyperparameters["weight_decay"],
    warmup_ratio=best_run.hyperparameters["warmup_ratio"],
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    logging_dir="./logs_best",
    save_total_limit=2,
    bf16=True
)

final_trainer = Trainer(
    model_init=helper.model_init,
    args=best_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=helper.tokenizer,
    compute_metrics=helper.compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

final_trainer.train()
metrics = final_trainer.evaluate()
print("Final evaluation metrics:", metrics)

final_trainer.save_model("./manipulation_classifier_best")


Map: 100%|██████████| 5775/5775 [00:00<00:00, 10558.67 examples/s]
/var/folders/cn/4_gky4xj72v72pn40z8_0vh80000gn/T/ipykernel_97326/3822643580.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2025-04-25 20:47:02,702] A new study created in memory with name: no-name-2b26d521-9733-454f-a71d-557528a920ef
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for 

Epoch,Training Loss,Validation Loss


[W 2025-04-25 21:08:16,905] Trial 0 failed with parameters: {'learning_rate': 1.8252791162085776e-05, 'per_device_train_batch_size': 16, 'num_train_epochs': 6, 'weight_decay': 0.10598152672780227, 'warmup_ratio': 0.17709049315077444} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/vittoriadiachenko/PycharmProjects/knowledge_manipulation/.venv/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/Users/vittoriadiachenko/PycharmProjects/knowledge_manipulation/.venv/lib/python3.10/site-packages/transformers/integrations/integration_utils.py", line 254, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/Users/vittoriadiachenko/PycharmProjects/knowledge_manipulation/.venv/lib/python3.10/site-packages/transformers/trainer.py", line 2245, in train
    return inner_training_loop(
  File "/Users/vittoriadiachenko/PycharmProjects/knowl

KeyboardInterrupt: 

In [25]:
uk_df = pd.read_csv("wikipedia/uk_holdout.csv")

In [26]:
uk_df.columns

Index(['Unnamed: 0', 'wiki_db', 'event_comment', 'event_user_text_historical',
       'event_user_seconds_since_previous_revision', 'revision_id',
       'page_title', 'page_revision_count', 'revision_text_bytes_diff',
       'revision_is_identity_reverted', 'event_timestamp',
       'revision_parent_id', 'revision_first_identity_reverting_revision_id',
       'reverting_revision_is_reverted_revision', 'is_reverted_by_good_user',
       'is_mobile_edit', 'is_mobile_web_edit', 'is_visualeditor',
       'is_wikieditor', 'is_mobile_app_edit', 'is_android_app_edit',
       'is_ios_app_edit', 'texts_removed', 'texts_insert', 'texts_change',
       'actions', 'is_reverted', 'user', 'event_date'],
      dtype='object')

In [28]:
import pandas as pd
from helpers.text_classifier_helper import TextClassifierHelper

helper = TextClassifierHelper("finetuned-manipulation-text-classifier")

df = pd.read_csv("uk_holdout_with_all_manip_features_and_logits.csv")

df["texts_insert"] = df["texts_insert"].apply(helper.join_text_list)
df["texts_removed"] = df["texts_removed"].apply(helper.join_text_list)

insert_df = helper.classify_texts(df["texts_insert"], prefix="manip_insert")
remove_df = helper.classify_texts(df["texts_removed"], prefix="manip_remove")

df["texts_change"] = df["texts_change"].apply(helper.join_changes)
df["joined_texts_change"] = df["texts_change"].apply(lambda changes: "\n".join(changes) if isinstance(changes, list) else "")

change_df = helper.classify_texts(df["joined_texts_change"], prefix="manip_change")

df = pd.concat([df.reset_index(drop=True), insert_df, remove_df, change_df], axis=1)

df.to_csv("uk_holdout_with_all_manip_features.csv", index=False)

<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:11: SyntaxWarning: invalid decimal literal
Running manipulation classifier for manip_change: 100%|██████████| 831308/831308 [2:20:10<00:00, 98.85it/s]   
